In [1]:
%cd ../

d:\mlops\mlflow_practice


In [15]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import numpy as np

In [16]:
with open('assets/feature_store/pca_data.json', 'r') as f:
    combined = json.load(f)
    
metadata = combined['metadata']
print(metadata["description"])
data = combined['data']


df = pd.DataFrame(data)
df.head()

this is pca data from raw


,0,1,target
0,-2.264703,0.480027,0
1,-2.080961,-0.674134,0
2,-2.364229,-0.341908,0
3,-2.299384,-0.597395,0
4,-2.389842,0.646835,0


In [17]:
x= df.drop(columns=["target"])
y= df["target"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [19]:
X_train.shape

(105, 2)

In [36]:
from flowerClassification.config import config
import flowerClassification.utils as utils

In [37]:
current_branch = utils.get_current_branch()
current_branch

head_commit = utils.get_current_commit_id()
head_commit

'9d5652ee863aaf2e6baf32885082b6f949c46515'

In [38]:
config

Box({'prep': {'pca_comps_count': 2}, 'logistic_regression': {'C': 1.0}, 'desition_tree': {'max_depth': 10}})

In [39]:
# Initialize the models
log_reg = LogisticRegression(C = config.logistic_regression.C)
dec_tree = DecisionTreeClassifier(max_depth = config.desition_tree.max_depth )

# Predict and evaluate the models
models = {
    'Logistic Regression': log_reg,
    'Decision Tree': dec_tree
}

In [43]:
mlflow.set_experiment("flowerClassification")


scores={
    
}

for model_name, model in models.items():
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    accuracy = report['accuracy']
    f1_score = report['weighted avg']["f1-score"]
    
    scores={
        **scores,
        f"{model_name}_accuracy":accuracy,
        f"{model_name}_f1_score":f1_score
    }
    
    print(report)    
    print(f"Model: {model_name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("-" * 60)



{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 19.0}, '1': {'precision': 0.9090909090909091, 'recall': 0.7692307692307693, 'f1-score': 0.8333333333333334, 'support': 13.0}, '2': {'precision': 0.8, 'recall': 0.9230769230769231, 'f1-score': 0.8571428571428571, 'support': 13.0}, 'accuracy': 0.9111111111111111, 'macro avg': {'precision': 0.903030303030303, 'recall': 0.8974358974358975, 'f1-score': 0.8968253968253969, 'support': 45.0}, 'weighted avg': {'precision': 0.915959595959596, 'recall': 0.9111111111111111, 'f1-score': 0.9105820105820107, 'support': 45.0}}
Model: Logistic Regression
Accuracy: 0.9111111111111111
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.91      0.77      0.83        13
           2       0.80      0.92      0.86        13

    accuracy                           0.91        45
   macro avg       0.90      0.90      0.90        45
weighted 

In [40]:
params = {
    "pca_comps" : config.prep.pca_comps_count,
    "lr_C":config.logistic_regression.C,
    "desition_tree_max_depth":config.desition_tree.max_depth
}

In [44]:
scores

{'Logistic Regression_accuracy': 0.9111111111111111,
 'Logistic Regression_f1_score': 0.9105820105820107,
 'Decision Tree_accuracy': 0.9333333333333333,
 'Decision Tree_f1_score': 0.9332345679012346}

In [45]:
current_branch,head_commit

('main', '9d5652ee863aaf2e6baf32885082b6f949c46515')

In [46]:
with mlflow.start_run(run_name="koko",tags={"branch":current_branch , 'commit_id':head_commit}) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(scores)